In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')/255.0
        self.test_x_f1 = np.load('../testImage_canny.npy')/255.0
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')/255.0
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')/255.0

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2= tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                   self.x_f1: batch_x_f1_test,
                                                                   self.x_f2: batch_x_f2_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [3]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 13.76, total 50 epochs ptime: 706.34
#  [*] Training finished!
#  [*] Best Epoch:  32 , Accuracy:  0.8299999833106995
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-32
#  [*] Finished testing Best Epoch: 32 , accuracy:  0.8299999833106995 

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 96)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32

Epoch: [ 3] [  15/  40] time: 35.6187, loss: 0.04833284
Epoch: [ 3] [  16/  40] time: 35.9456, loss: 0.01606773
Epoch: [ 3] [  17/  40] time: 36.2712, loss: 0.02584805
Epoch: [ 3] [  18/  40] time: 36.5969, loss: 0.01328489
Epoch: [ 3] [  19/  40] time: 36.9211, loss: 0.02605097
Epoch: [ 3] [  20/  40] time: 37.2464, loss: 0.04283865
Epoch: [ 3] [  21/  40] time: 37.5711, loss: 0.09786599
Epoch: [ 3] [  22/  40] time: 37.8953, loss: 0.01504988
Epoch: [ 3] [  23/  40] time: 38.2205, loss: 0.20082149
Epoch: [ 3] [  24/  40] time: 38.5468, loss: 0.05647022
Epoch: [ 3] [  25/  40] time: 38.8711, loss: 0.03794533
Epoch: [ 3] [  26/  40] time: 39.1967, loss: 0.05582380
Epoch: [ 3] [  27/  40] time: 39.5229, loss: 0.03479865
Epoch: [ 3] [  28/  40] time: 39.8475, loss: 0.07733450
Epoch: [ 3] [  29/  40] time: 40.1741, loss: 0.03195048
Epoch: [ 3] [  30/  40] time: 40.5001, loss: 0.06161539
Epoch: [ 3] [  31/  40] time: 40.8249, loss: 0.13808812
Epoch: [ 3] [  32/  40] time: 41.1504, loss: 0.0

Epoch: [ 6] [  38/  40] time: 85.2933, loss: 0.00872104
Epoch: [ 6] [  39/  40] time: 85.6180, loss: 0.00461467
[6/50] - ptime: 13.6968 loss: 0.01143453 acc: 0.22000 lr: 0.00100000
Epoch: [ 7] [   0/  40] time: 86.9481, loss: 0.00122719
Epoch: [ 7] [   1/  40] time: 87.2740, loss: 0.00437656
Epoch: [ 7] [   2/  40] time: 87.5997, loss: 0.02664799
Epoch: [ 7] [   3/  40] time: 87.9256, loss: 0.00340236
Epoch: [ 7] [   4/  40] time: 88.2526, loss: 0.00621296
Epoch: [ 7] [   5/  40] time: 88.5795, loss: 0.00010185
Epoch: [ 7] [   6/  40] time: 88.9029, loss: 0.01165257
Epoch: [ 7] [   7/  40] time: 89.2291, loss: 0.01181485
Epoch: [ 7] [   8/  40] time: 89.5554, loss: 0.00065602
Epoch: [ 7] [   9/  40] time: 89.8815, loss: 0.02483445
Epoch: [ 7] [  10/  40] time: 90.2073, loss: 0.01052967
Epoch: [ 7] [  11/  40] time: 90.5337, loss: 0.00418014
Epoch: [ 7] [  12/  40] time: 90.8581, loss: 0.01577828
Epoch: [ 7] [  13/  40] time: 91.1846, loss: 0.00859442
Epoch: [ 7] [  14/  40] time: 91.51

Epoch: [10] [  18/  40] time: 134.9899, loss: 0.00488576
Epoch: [10] [  19/  40] time: 135.3183, loss: 0.01900413
Epoch: [10] [  20/  40] time: 135.6459, loss: 0.00008195
Epoch: [10] [  21/  40] time: 135.9718, loss: 0.01392921
Epoch: [10] [  22/  40] time: 136.2998, loss: 0.00053158
Epoch: [10] [  23/  40] time: 136.6281, loss: 0.04617441
Epoch: [10] [  24/  40] time: 136.9536, loss: 0.00118352
Epoch: [10] [  25/  40] time: 137.2801, loss: 0.03439321
Epoch: [10] [  26/  40] time: 137.6044, loss: 0.00069453
Epoch: [10] [  27/  40] time: 137.9291, loss: 0.10007116
Epoch: [10] [  28/  40] time: 138.2554, loss: 0.02756255
Epoch: [10] [  29/  40] time: 138.5806, loss: 0.01760101
Epoch: [10] [  30/  40] time: 138.9054, loss: 0.01074014
Epoch: [10] [  31/  40] time: 139.2317, loss: 0.04595516
Epoch: [10] [  32/  40] time: 139.5575, loss: 0.02846934
Epoch: [10] [  33/  40] time: 139.8828, loss: 0.01870389
Epoch: [10] [  34/  40] time: 140.2092, loss: 0.02450853
Epoch: [10] [  35/  40] time: 1

Epoch: [13] [  39/  40] time: 184.0435, loss: 0.02028724
[13/50] - ptime: 13.7195 loss: 0.00916840 acc: 0.70000 lr: 0.00090000
Epoch: [14] [   0/  40] time: 185.3859, loss: 0.00137477
Epoch: [14] [   1/  40] time: 185.7144, loss: 0.00019669
Epoch: [14] [   2/  40] time: 186.0421, loss: 0.00004319
Epoch: [14] [   3/  40] time: 186.3690, loss: 0.00069968
Epoch: [14] [   4/  40] time: 186.6962, loss: 0.00017257
Epoch: [14] [   5/  40] time: 187.0213, loss: 0.00167026
Epoch: [14] [   6/  40] time: 187.3496, loss: 0.00029403
Epoch: [14] [   7/  40] time: 187.6748, loss: 0.00023232
Epoch: [14] [   8/  40] time: 187.9997, loss: 0.00062932
Epoch: [14] [   9/  40] time: 188.3260, loss: 0.00146761
Epoch: [14] [  10/  40] time: 188.6510, loss: 0.00032082
Epoch: [14] [  11/  40] time: 188.9760, loss: 0.00397556
Epoch: [14] [  12/  40] time: 189.3025, loss: 0.00495457
Epoch: [14] [  13/  40] time: 189.6282, loss: 0.00407572
Epoch: [14] [  14/  40] time: 189.9544, loss: 0.02455784
Epoch: [14] [  15/

...

Epoch: [34] [  39/  40] time: 480.1975, loss: 0.00001946
[34/50] - ptime: 13.7287 loss: 0.00029858 acc: 0.79000 lr: 0.00072900
Epoch: [35] [   0/  40] time: 481.5304, loss: 0.00005970
Epoch: [35] [   1/  40] time: 481.8530, loss: 0.00002196
Epoch: [35] [   2/  40] time: 482.1792, loss: 0.00020302
Epoch: [35] [   3/  40] time: 482.5058, loss: 0.00001540
Epoch: [35] [   4/  40] time: 482.8314, loss: 0.00000452
Epoch: [35] [   5/  40] time: 483.1582, loss: 0.00000141
Epoch: [35] [   6/  40] time: 483.4849, loss: 0.00000159
Epoch: [35] [   7/  40] time: 483.8094, loss: 0.00001007
Epoch: [35] [   8/  40] time: 484.1361, loss: 0.00000879
Epoch: [35] [   9/  40] time: 484.4636, loss: 0.00001630
Epoch: [35] [  10/  40] time: 484.7895, loss: 0.00000200
Epoch: [35] [  11/  40] time: 485.1150, loss: 0.00000039
Epoch: [35] [  12/  40] time: 485.4429, loss: 0.00002958
Epoch: [35] [  13/  40] time: 485.7697, loss: 0.00006306
Epoch: [35] [  14/  40] time: 486.0965, loss: 0.00003218
Epoch: [35] [  15/

Epoch: [38] [  18/  40] time: 529.7949, loss: 0.00000800
Epoch: [38] [  19/  40] time: 530.1216, loss: 0.00002910
Epoch: [38] [  20/  40] time: 530.4467, loss: 0.00001720
Epoch: [38] [  21/  40] time: 530.7731, loss: 0.00000576
Epoch: [38] [  22/  40] time: 531.0997, loss: 0.00001894
Epoch: [38] [  23/  40] time: 531.4267, loss: 0.00004982
Epoch: [38] [  24/  40] time: 531.7525, loss: 0.00000029
Epoch: [38] [  25/  40] time: 532.0816, loss: 0.00018273
Epoch: [38] [  26/  40] time: 532.4088, loss: 0.00001067
Epoch: [38] [  27/  40] time: 532.7346, loss: 0.00007505
Epoch: [38] [  28/  40] time: 533.0653, loss: 0.00003003
Epoch: [38] [  29/  40] time: 533.3908, loss: 0.00000160
Epoch: [38] [  30/  40] time: 533.7176, loss: 0.00002896
Epoch: [38] [  31/  40] time: 534.0412, loss: 0.00000421
Epoch: [38] [  32/  40] time: 534.3679, loss: 0.00000068
Epoch: [38] [  33/  40] time: 534.6941, loss: 0.00000321
Epoch: [38] [  34/  40] time: 535.0204, loss: 0.00001220
Epoch: [38] [  35/  40] time: 5

Epoch: [41] [  39/  40] time: 579.0363, loss: 0.00000589
[41/50] - ptime: 13.7514 loss: 0.00003365 acc: 0.83000 lr: 0.00065610
Epoch: [42] [   0/  40] time: 580.3873, loss: 0.00000228
Epoch: [42] [   1/  40] time: 580.7125, loss: 0.00000047
Epoch: [42] [   2/  40] time: 581.0378, loss: 0.00000598
Epoch: [42] [   3/  40] time: 581.3650, loss: 0.00000389
Epoch: [42] [   4/  40] time: 581.6913, loss: 0.00000292
Epoch: [42] [   5/  40] time: 582.0171, loss: 0.00001041
Epoch: [42] [   6/  40] time: 582.3438, loss: 0.00002009
Epoch: [42] [   7/  40] time: 582.6705, loss: 0.00001063
Epoch: [42] [   8/  40] time: 582.9961, loss: 0.00000912
Epoch: [42] [   9/  40] time: 583.3233, loss: 0.00000122
Epoch: [42] [  10/  40] time: 583.6492, loss: 0.00000052
Epoch: [42] [  11/  40] time: 583.9753, loss: 0.00000583
Epoch: [42] [  12/  40] time: 584.3031, loss: 0.00001536
Epoch: [42] [  13/  40] time: 584.6304, loss: 0.00002000
Epoch: [42] [  14/  40] time: 584.9570, loss: 0.00002045
Epoch: [42] [  15/

Epoch: [45] [  18/  40] time: 628.4873, loss: 0.00000090
Epoch: [45] [  19/  40] time: 628.8138, loss: 0.00001790
Epoch: [45] [  20/  40] time: 629.1414, loss: 0.00000211
Epoch: [45] [  21/  40] time: 629.4689, loss: 0.00000632
Epoch: [45] [  22/  40] time: 629.7945, loss: 0.00000177
Epoch: [45] [  23/  40] time: 630.1204, loss: 0.00000328
Epoch: [45] [  24/  40] time: 630.4460, loss: 0.00000034
Epoch: [45] [  25/  40] time: 630.7718, loss: 0.00000176
Epoch: [45] [  26/  40] time: 631.0979, loss: 0.00001761
Epoch: [45] [  27/  40] time: 631.4227, loss: 0.00000010
Epoch: [45] [  28/  40] time: 631.7488, loss: 0.00000071
Epoch: [45] [  29/  40] time: 632.0764, loss: 0.00000050
Epoch: [45] [  30/  40] time: 632.4021, loss: 0.00000490
Epoch: [45] [  31/  40] time: 632.7291, loss: 0.00008742
Epoch: [45] [  32/  40] time: 633.0554, loss: 0.00000071
Epoch: [45] [  33/  40] time: 633.3826, loss: 0.00000037
Epoch: [45] [  34/  40] time: 633.7097, loss: 0.00000898
Epoch: [45] [  35/  40] time: 6

Epoch: [48] [  39/  40] time: 677.5845, loss: 0.00000106
[48/50] - ptime: 13.7298 loss: 0.00001503 acc: 0.81000 lr: 0.00065610
Epoch: [49] [   0/  40] time: 678.9636, loss: 0.00002140
Epoch: [49] [   1/  40] time: 679.2906, loss: 0.00000017
Epoch: [49] [   2/  40] time: 679.6174, loss: 0.00004819
Epoch: [49] [   3/  40] time: 679.9438, loss: 0.00000598
Epoch: [49] [   4/  40] time: 680.2716, loss: 0.00002295
Epoch: [49] [   5/  40] time: 680.5985, loss: 0.00000744
Epoch: [49] [   6/  40] time: 680.9251, loss: 0.00001125
Epoch: [49] [   7/  40] time: 681.2542, loss: 0.00000619
Epoch: [49] [   8/  40] time: 681.5820, loss: 0.00000015
Epoch: [49] [   9/  40] time: 681.9082, loss: 0.00000227
Epoch: [49] [  10/  40] time: 682.2355, loss: 0.00000401
Epoch: [49] [  11/  40] time: 682.5618, loss: 0.00000391
Epoch: [49] [  12/  40] time: 682.8870, loss: 0.00000230
Epoch: [49] [  13/  40] time: 683.2135, loss: 0.00000071
Epoch: [49] [  14/  40] time: 683.5422, loss: 0.00000056
Epoch: [49] [  15/

In [4]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放